In [53]:
import pandas as pd
import os
import logging
import sys
import pyodbc

In [54]:
nombre_archivo=r'C:\Users\user\Desktop\PRUEBA_TECNICA\Ingeniero_datos_p_tecnica\data\raw'

In [55]:
def conf_logging (log_archivo:str):#conf_logging (log_archivo):

    logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_archivo, mode='a',encoding="utf-8"),  # Guardar log en archivo
        logging.StreamHandler(),  # También quiero imprimir log en la consola
    ],
    )
    logger = logging.getLogger()
    return logger

In [56]:
try:
    conn = pyodbc.connect(
        "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
        "SERVER=localhost;"
        "DATABASE=pipeline_db;" 
        "PORT=5433;"
        "UID=postgres;"
        "PWD=postgres"
    )
    logging.info("Conexión a la base de datos sumi exitosa.")
except pyodbc.Error as e:
    logging.error(f"Error al conectar con la base de datos sumi: {e}")
    raise

2025-07-28 19:46:05,660 - INFO - Conexión a la base de datos sumi exitosa.


In [57]:
logger = conf_logging("notbook_log")

In [58]:
def consultar(conn,consulta):
    try:
        cursor=conn.cursor()
        query=consulta
        cursor.execute(query)
        get=cursor.fetchall()
        columnas=[column[0] for column in cursor.description]
        df_excel=pd.DataFrame.from_records(get,columns=columnas)#.astype(str)
        return df_excel
    except Exception as e:
        logging.info(f"Ocurrio un error al consultar ({consulta}): {e}")
        return None

In [59]:
nombre_archivo_2=os.path.join(nombre_archivo, 'data_jobs.csv')
try:
    df_data=pd.read_csv(f'{nombre_archivo_2}', sep=',',low_memory=False)
except Exception as e:
    logger.error(f"Error al cargar el archivo: {e}")
    raise

In [96]:
def mapeo_tipo_dato(tipos, muestra=None):
        if pd.api.types.is_integer_dtype(tipos):
            return 'INTEGER'
        elif pd.api.types.is_float_dtype(tipos):
            return 'FLOAT'
        elif pd.api.types.is_bool_dtype(tipos):
            return "BOOLEAN"
        elif pd.api.types.is_datetime64_any_dtype(tipos):
            return "TIMESTAMP"
        elif pd.api.types.is_object_dtype(tipos) and isinstance(muestra, list):
            return "TEXT"
        elif pd.api.types.is_object_dtype(tipos) and isinstance(muestra, dict):
            return "JSONB"
        elif pd.api.types.is_string_dtype(tipos):
            return "TEXT"
        else:
            return "TEXT"

def crear_tabla_dinamica(conn,df, tabla):
    cursor = conn.cursor()
    columnas = []
    
    for col in df.columns:
        tipo_pg = mapeo_tipo_dato(df[col].dtype, df[col].dropna().iloc[0] if not df[col].dropna().empty else None)
        col_limpio = col.lower().strip().replace(" ", "_")
        columnas.append(f'"{col_limpio}" {tipo_pg}')
    
    columnas_sql = ", ".join(columnas)
    sql = f"""
        CREATE TABLE IF NOT EXISTS {tabla} (
            {columnas_sql}
        );
    """
    
    cursor.execute(sql)
    conn.commit()
    return sql

In [ ]:
tabla='data_jobs_raw'
a=crear_tabla_dinamica(conn, df_data, tabla)

In [80]:
df=consultar(conn, "SELECT * FROM data_jobs_raw;")

2025-07-28 19:54:11,077 - INFO - Ocurrio un error al consultar (SELECT * FROM data_jobs_raw;): ('42P01', '[42P01] ERROR: relation "data_jobs_raw" does not exist;\nError while executing the query (1) (SQLExecDirectW)')


In [77]:
print(a)


        CREATE TABLE IF NOT EXISTS data_jobs_raw (
            "job_title_short" VARCHAR(255), "job_title" VARCHAR(255), "job_location" VARCHAR(255), "job_via" VARCHAR(255), "job_schedule_type" VARCHAR(255), "job_work_from_home" BOOLEAN, "search_location" VARCHAR(255), "job_posted_date" VARCHAR(255), "job_no_degree_mention" BOOLEAN, "job_health_insurance" BOOLEAN, "job_country" VARCHAR(255), "salary_rate" VARCHAR(255), "salary_year_avg" FLOAT, "salary_hour_avg" FLOAT, "company_name" VARCHAR(255), "job_skills" VARCHAR(255), "job_type_skills" VARCHAR(255)
        );
    


In [ ]:

import psycopg2
try:
    conn2 = psycopg2.connect(
        host="localhost",
        database="pipeline_db",
        user="postgres",
        password="postgres",
        port=5433
    )
    logging.info("Conexión a la base de datos exitosa.")
except psycopg2.Error as e:
    logging.error(f"Error al conectar con la base de datos: {e}")
    raise



import psycopg2
import io

def bulk_insert_copy(df, tabla, conn):
    cursor = conn.cursor()
    crear_tabla_dinamica(conn, df_data, tabla)

    df = df.replace({r'\t': ' ', r'\n': ' ', r'\r': ' '}, regex=True)
    df = df.applymap(lambda x: str(x).replace('"', '""') if isinstance(x, str) else x)

    output = io.StringIO() #moviendome en la memoria...
    df.to_csv(output, sep='\t', header=False, index=False, na_rep='\\N')
    output.seek(0)

    try:
        cursor.copy_from(output, tabla, sep='\t', null='\\N')
        conn.commit()
        logger.info("Inserción exitosa.")
    except Exception as e:
        logger.error(f"Error en copy_from: {e}")
        conn.rollback()
        raise
    finally:
        cursor.close()

bulk_insert_copy(df_data, tabla, conn2)


2025-07-28 20:46:12,667 - INFO - Conexión a la base de datos exitosa.
C:\Users\user\AppData\Local\Temp\ipykernel_28084\448116207.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace('"', '""') if isinstance(x, str) else x)


Inserción exitosa.


In [92]:
longitudes = df_data.select_dtypes(include='object').applymap(lambda x: len(str(x)) if x else 0).max()
print("Longitud máxima por columna:\n", longitudes.sort_values(ascending=False))

C:\Users\user\AppData\Local\Temp\ipykernel_28084\3801102422.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  longitudes = df_data.select_dtypes(include='object').applymap(lambda x: len(str(x)) if x else 0).max()


Longitud máxima por columna:
 company_name         939
job_type_skills      651
job_skills           536
job_title            113
job_via              113
job_location          95
job_schedule_type     37
search_location       33
job_country           33
job_title_short       25
job_posted_date       19
salary_rate            5
dtype: int64


In [82]:
print(columnas)

"job_title_short", "job_title", "job_location", "job_via", "job_schedule_type", "job_work_from_home", "search_location", "job_posted_date", "job_no_degree_mention", "job_health_insurance", "job_country", "salary_rate", "salary_year_avg", "salary_hour_avg", "company_name", "job_skills", "job_type_skills"


In [113]:
cursor = conn.cursor()
cursor.execute("drop table if exists data_jobs_raw;")
conn.commit()

In [52]:
cursor.close()
conn.close()